In [14]:
import re
import pandas as pd
import os
from datetime import datetime
# Function to extract date and name from the filename
def extract_date_and_name(filename):
    # match = re.match(r'(\d{8})_([A-Za-z]+)', filename)
    match = re.match(r'(\d{8})(.*?)(?=\d|$)', filename)
    if match:
        date, name = match.groups()
        modified_name = re.sub(r'\s+', '', name)  # Remove all spaces
        modified_name = re.sub(r'\.', '_', modified_name) 
        return date, name
    return None, None

directory = './data_txt/pdftotext_upright_pages/'

In [110]:
date_pattern = re.compile(r'\b(0?[1-9]|1[0-2])/(0?[1-9]|[12][0-9]|3[01])/([0-9]{4})\b')
transaction_pattern = re.compile(r'(?:^|[\t ])(s|P|S|p|E)[\t ]+(?:\d+|\(partial\))')
amount_range_pattern = re.compile(r'1 -')
amount_number_pattern = re.compile(r'\$(\d+\.?\d*) -')
token_pattern = re.compile(r'\(([A-Za-z]+)\)')

def accept_line(line):
    statement = date_pattern.search(line) and transaction_pattern.search(line) \
        and amount_range_pattern.search(line)
    return statement

def find_token(line):
    token = token_pattern.search(line)
    if (not token is None):
        return token.group(1)
    return None

def find_token_by_index(lines, line_ind, index):
    next_line_ind = line_ind + index
    if next_line_ind < len(lines):
        next_line = lines[next_line_ind]
        if not accept_line(next_line):
            token = find_token(lines[next_line_ind])
            if (not token is None):
                return token
    return None

amounts_set = set()
transaction_type_set = set()
token_set = set()
count = 0
token_count = 0
rows = []

for filename in os.listdir(directory):
    # Check if the file has a .md extension
    if filename.endswith('.txt'):
        date, name = extract_date_and_name(filename)

        # Read the file
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            file_content=file.read()
            # Split the file into lines
        lines = file_content.strip().split('\n')

        # Iterate over the lines
        for line_ind, line in enumerate(lines):

            # Filter out lines that contain the pattern
            if accept_line(line):
                count += 1
                # Recover transaction date
                dates = date_pattern.findall(line)
                if len(dates) != 2:
                    transaction_date = dates[0]
                else:
                    transaction_date = dates[1]
                transaction_date = transaction_date[2] + transaction_date[0] \
                    + transaction_date[1]

                # Recover amount
                amount = amount_number_pattern.search(line.replace(',', '')).group(1)
                amounts_set.add(amount)

                # Recover transaction type
                transaction_type = transaction_pattern.search(line).group(1)
                if (transaction_type =='s'):
                    transaction_type = 'S'
                elif (transaction_type =='p'):
                    transaction_type = 'P'
                transaction_type_set.add(transaction_type)

                # Recover residual of the line:
                name_residual = line[:transaction_pattern.search(line).start()].strip()
                
                # Recover token
                token = find_token(name_residual)
                max_index = 1
                transaction_name = name_residual
                if (token is None):
                    while((token is None) and (max_index < 5)):
                        token = find_token_by_index(lines, line_ind, max_index)
                        max_index += 1
                if (not token is None):
                    token_count += 1
                    for i in range(max_index):
                        transaction_name += ' ' + lines[line_ind + i + 1]
                else:
                    token = 'N/A'
                    
                # Save the transaction
                rows.append({
                    'Stock_name': transaction_name,
                    'Ticker': token,
                    'Type': transaction_type,
                    'Date': transaction_date,
                    'Min_amount': amount,
                    'Trader_name': name,
                    'Document_date': date
                })

df = pd.DataFrame(rows)
print(token_count)
print(count)

29406
43824


In [113]:
df[df['Ticker'] != 'N/A']

,Stock_name,Ticker,Type,Date,Min_Amount,Pidor_name,Document_date
0,"Home Depot, Inc. (HD) [ST] ...",HD,P,20210405,1001,LloydDoggett,20210407
2,"PPg Industries, Inc. (PPg) [ST] ...",PPg,P,20210405,1001,LloydDoggett,20210407
4,JT AECOM (ACM) [ST] ...,ACM,S,20201201,1001,SusieLee,20201208
5,JT Aptiv PLC Ordinary Shares (APTV) ...,APTV,P,20201101,1001,SusieLee,20201208
6,JT Aptiv PLC Ordinary Shares (APTV) ...,APTV,P,20201101,1001,SusieLee,20201208
...,...,...,...,...,...,...,...
43811,Nisource Inc (NI) ID Owner Asset ...,NI,P,20150826,1001,LoisFrankel,20150925
43812,omnicom Group Inc. (oMC) FI...,oMC,P,20150826,1001,LoisFrankel,20150925
43813,schlumberger N.V. (sLb) FIL...,sLb,P,2015099,1001,LoisFrankel,20150925
43814,"TECo Energy, Inc. (TE) FILI...",TE,S,2015099,1001,LoisFrankel,20150925
